In [10]:

import random
import numpy as np

# Define constants
POP_SIZE = 20
GENERATIONS = 50
GENE_LENGTH = 10  # Length of a chromosome
TERMINALS = ['x', '1', '2']  # Terminal symbols
FUNCTIONS = ['+', '-', '*']  # Function symbols

# Generate random chromosome
def generate_chromosome():
    return [random.choice(FUNCTIONS + TERMINALS) for _ in range(GENE_LENGTH)]

# Decode chromosome into an expression
def decode_chromosome(chromosome):
    stack = []
    for gene in chromosome:
        if gene in FUNCTIONS and len(stack) >= 2:
            b = stack.pop()
            a = stack.pop()
            stack.append(f"({a} {gene} {b})")
        else:
            stack.append(gene)
    return stack[0] if stack else ""

# Evaluate fitness
def evaluate_fitness(expression, x_values, y_target):
    try:
        y_pred = [eval(expression.replace('x', str(x))) for x in x_values]
        return -np.sum((np.array(y_pred) - np.array(y_target))**2)  # Negative error
    except:
        return float('-inf')

# Genetic operators
def mutate(chromosome):
    idx = random.randint(0, GENE_LENGTH-1)
    chromosome[idx] = random.choice(FUNCTIONS + TERMINALS)
    return chromosome

def crossover(parent1, parent2):
    idx = random.randint(1, GENE_LENGTH-1)
    return parent1[:idx] + parent2[idx:], parent2[:idx] + parent1[idx:]

# Main GEP algorithm
x_values = np.linspace(-10, 10, 20)
y_target = x_values**2 + 2*x_values + 1

population = [generate_chromosome() for _ in range(POP_SIZE)]

for gen in range(GENERATIONS):
    fitness = [evaluate_fitness(decode_chromosome(chrom), x_values, y_target) for chrom in population]
    sorted_population = [x for _, x in sorted(zip(fitness, population), reverse=True)]
    population = sorted_population[:POP_SIZE//2]  # Selection

    # Crossover and mutation
    offspring = []
    while len(offspring) < POP_SIZE//2:
        p1, p2 = random.sample(population, 2)
        c1, c2 = crossover(p1, p2)
        offspring.append(mutate(c1))
        offspring.append(mutate(c2))
    population = population + offspring

# Print the best solution
best_chromosome = sorted(population, key=lambda c: evaluate_fitness(decode_chromosome(c), x_values, y_target))[-1]
print("Best Solution:", decode_chromosome(best_chromosome))


Best Solution: ((x * (x + 1)) + (1 + x))
